# Soft SVM (Support vector machine) using Stochastic Gradient Descent
# min(w) (0.5*||w||^2 + hinge_loss(w)) 
# where hinge_loss((w, b),(x, y)) = 1/m(summation_over_m(max{0, 1 − y(hw, xi + b)}))


## Step 1: Import All required Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

## Step 2: Read CSV file (For this i have uploded the file on google colab) 

In [2]:
df = pd.read_csv('file.csv')
print(df.shape,df)

(2665, 6)       Temperature   Humidity       Light          CO2  HumidityRatio  Occupancy
0       23.700000  26.272000  585.200000   749.200000       0.004764          1
1       23.718000  26.290000  578.400000   760.400000       0.004773          1
2       23.730000  26.230000  572.666667   769.666667       0.004765          1
3       23.722500  26.125000  493.750000   774.750000       0.004744          1
4       23.754000  26.200000  488.600000   779.000000       0.004767          1
...           ...        ...         ...          ...            ...        ...
2660    24.290000  25.700000  808.000000  1150.250000       0.004829          1
2661    24.330000  25.736000  809.800000  1129.200000       0.004848          1
2662    24.330000  25.700000  817.000000  1125.800000       0.004841          1
2663    24.356667  25.700000  813.000000  1123.000000       0.004849          1
2664    24.408333  25.681667  798.000000  1124.000000       0.004860          1

[2665 rows x 6 columns]


## Step 3: Store the features and target in X , Y seperately and since target has value 0,1 change label 0 to -1 

In [3]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1:].values
print(X.shape,X)
Y[Y==0]=-1
print(Y.shape,Y)

(2665, 5) [[2.37000000e+01 2.62720000e+01 5.85200000e+02 7.49200000e+02
  4.76416302e-03]
 [2.37180000e+01 2.62900000e+01 5.78400000e+02 7.60400000e+02
  4.77266099e-03]
 [2.37300000e+01 2.62300000e+01 5.72666667e+02 7.69666667e+02
  4.76515255e-03]
 ...
 [2.43300000e+01 2.57000000e+01 8.17000000e+02 1.12580000e+03
  4.84075873e-03]
 [2.43566667e+01 2.57000000e+01 8.13000000e+02 1.12300000e+03
  4.84855928e-03]
 [2.44083333e+01 2.56816667e+01 7.98000000e+02 1.12400000e+03
  4.86020770e-03]]
(2665, 1) [[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


## Step 4: Processing the Data and Splitting


In [4]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
X_scale

array([[0.83168317, 0.44513204, 0.34479305, 0.33003334, 0.70421197],
       [0.8359604 , 0.44705255, 0.34078657, 0.34152347, 0.70830847],
       [0.83881188, 0.44065084, 0.33740855, 0.35103018, 0.70468898],
       ...,
       [0.98138614, 0.38410243, 0.48136692, 0.71638882, 0.74113545],
       [0.98772277, 0.38410243, 0.47901016, 0.71351629, 0.74489576],
       [1.        , 0.38214635, 0.47017234, 0.71454219, 0.75051096]])

A. Split the data in 70,30 for training and testing the model 

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_scale, Y, test_size=0.3, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1865, 5) (800, 5) (1865, 1) (800, 1)


B. Split the data in 80,20 for training and testing the model 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2132, 5) (533, 5) (2132, 1) (533, 1)


C. Split the data in 90,10 for training and testing the model 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2398, 5) (267, 5) (2398, 1) (267, 1)


## Step 5: Defining class for optimizing the weight vector w using the stochastic gradient descent.
### In stochastic gradient descent algorithm we minimise the true risk by creating a function that calculates the hinge loss for examples and then we update w using the sub gradient.
### As hinge loss is given by :
## hinge_loss((w, b),(x, y)) = max{0, 1 − y(hw, xi + b)}
## Also the gradients are upgraded as:
### if(yi<w,xi> > 1) then w=summation(-lambda*yi*xi) and b=0
### else w=summation(-lambda*yi*xi)+yi*xi and b=-yi
## also we can say that 1/(c) is our lambda regularization paprameter.



In [8]:
# Defining calss
class my_defined_SVM:
  # defining constructor and providing the values of parameters
  def __init__(self, learning_rate=0.001, lambda_param_for_regularization=0.01, n_iterations=1000):
    self.lr = learning_rate
    self.lambda_param = lambda_param_for_regularization
    self.n = n_iterations
    self.w = None
    self.b = None
  # defining the fit model using stochastic gradient descent
  def fit_model(self, X, y):
    n_samples, n_features = X.shape
    self.w = np.zeros(n_features)
    self.b = 0
    # iterations for updation of parameters using stochastic gradient descent
    for i in range(self.n):
      for id,x_i in enumerate(X):
        # condition is the for checking the hinge loss
        condition = y[id] * (np.dot(x_i, self.w) - self.b) >= 1
        if condition:
          # updation of w and b using gradient
          self.w -= self.lr * (2 * self.lambda_param * self.w)
        else:
          # updation of w and b using gradient
          self.w -= self.lr * (2 * self.lambda_param * self.w - (x_i*y[id]))
          self.b -= self.lr * y[id]
  # defining the function for predicting the values
  def predict(self, X):
    product = np.dot(X, self.w) - self.b
    return np.sign(product)

## Step 6: Creating the model object and passing different values of lambda regularization parameters.

In [9]:
# default values of lambda
model = my_defined_SVM()
model.fit_model(X_train,y_train)
print("For lambda 0.01")
print(model.w,model.b)

# increasing values of lambda
model1 = my_defined_SVM(lambda_param_for_regularization=0.1)
model1.fit_model(X_train,y_train)
print("For lambda 0.1")
print(model1.w,model1.b)

# decreasing values of lambda
model2 = my_defined_SVM(lambda_param_for_regularization=0.001)
model2.fit_model(X_train,y_train)
print("For lambda 0.001")
print(model2.w,model2.b)

For lambda 0.01
[-8.14644166e+00 -7.01676773e+00  2.55486281e+00  4.06875247e-01
 -9.92156414e-04] [19.588]
For lambda 0.1
[-1.16872627e+00 -1.03996127e+00  3.06486128e+00 -9.26646358e-02
 -1.45262831e-04] [29.191]
For lambda 0.001
[-3.25625593e+01 -2.25470523e+01  2.57129722e+00  1.35898536e+00
 -3.56106411e-03] [8.129]


## Step 7: Creating target Array and calculating the target values.

In [10]:
target = model.predict(X_test)
t = model.predict(X_train)

target1 = model1.predict(X_test)
t1 = model1.predict(X_train)

target2 = model2.predict(X_test)
t2 = model2.predict(X_train)

## Step 8: Checking for Accuracy. 

In [11]:
v1 = 0  
v2 = 0
c = 0
i=0
sv=0
for i in range(X_train.shape[0]):
  if y_train[i]*(np.dot(X_train[i], model.w) - model.b) >= 1:
    sv=sv+1    
for c in range( np.size( target ) ) :  
  if y_test[c] == target[c] :            
    v1 = v1 + 1
  c = c + 1
for i in range( np.size( t ) ) :
  if y_train[i] == t[i] :            
    v2 = v2 + 1
  i = i + 1
print("For lambda = 0.01, c=100")
print("Accuracy of Defined Model on test data :",(v1/c)*100)     
print("Accuracy of Defined Model on train data:",(v2/i)*100)   

For lambda = 0.01, c=100
Accuracy of Defined Model on test data : 94.00749063670412
Accuracy of Defined Model on train data: 95.7464553794829


In [12]:
v1 = 0  
v2 = 0
c = 0
i=0
sv1=0
for i in range(X_train.shape[0]):
  if y_train[i]*(np.dot(X_train[i], model1.w) - model1.b) >= 1:
    sv1=sv1+1
for c in range( np.size( target1 ) ) :  
  if y_test[c] == target1[c] :            
    v1 = v1 + 1
  c = c +1   
for i in range( np.size( t1 ) ) :
  if y_train[i] == t1[i] :            
    v2 = v2 + 1
  i = i + 1
print("For lambda = 0.1, c=10")
print("Accuracy of Defined Model on test data :",(v1/c)*100)     
print("Accuracy of Defined Model on train data:",(v2/i)*100)   

For lambda = 0.1, c=10
Accuracy of Defined Model on test data : 96.25468164794007
Accuracy of Defined Model on train data: 97.16430358632194


In [13]:
v1 = 0  
v2 = 0
c = 0
i=0
sv2=0
for i in range(X_train.shape[0]):
  if y_train[i]*(np.dot(X_train[i], model2.w) - model2.b) >= 1:
    sv2=sv2+1   
for c in range( np.size( target2 ) ) :  
  if y_test[c] == target2[c] :            
    v1 = v1 + 1
  c = c +1
for i in range( np.size( t2 ) ) :
  if y_train[i] == t2[i] :            
    v2 = v2 + 1
  i = i + 1
print("For lambda = 0.001, c=1000")
print("Accuracy of Defined Model on test data :",(v1/c)*100)     
print("Accuracy of Defined Model on train data:",(v2/i)*100)   

For lambda = 0.001, c=1000
Accuracy of Defined Model on test data : 94.3820224719101
Accuracy of Defined Model on train data: 95.95496246872393


# Hence we can see different results for different values of regularization factor.
